In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split 
from torch.nn import functional as F

 
import numpy as np
import matplotlib
import matplotlib.pyplot as mplot
from matplotlib.colors import ListedColormap
import os
import subprocess
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score, precision_score, recall_score,  precision_recall_fscore_support
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
 
import random
from sklearn.preprocessing import StandardScaler
from sklearn.tree import export_graphviz

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

import pydotplus
import seaborn as sns

from tabulate import tabulate 


In [ ]:
dataSetIndex = 6
needToMakePictureOfTrees = 0
''' ---------------------------------------------------------- '''
dataSetFilePath = ""
dataSetName = ""
dataSetResultDirectory = "./"

if(dataSetIndex == 0):
    dataSetFilePath = "./heartDisease/0_statLog_dataSet.csv"
    dataSetName = "SateLog_DataSet"
elif (dataSetIndex == 1):
    dataSetFilePath = "./heartDisease/1_heart_statlog_cleveland_hungary_final.csv"
    dataSetName = "ALL_StateLog_CleveLand_Hungary"
elif (dataSetIndex == 2):
    dataSetFilePath = "./heartDisease/2_cleveland.csv"
    dataSetName = "Cleveland"
elif (dataSetIndex == 3):
    dataSetFilePath = "./heartDisease/3_framingham.csv"
    dataSetName = "framingham"
elif (dataSetIndex == 4):
    dataSetFilePath = "./heartDisease/4_CardiacPrediction.xlsx"
    dataSetName = "CardiacPrediction"
elif (dataSetIndex == 5):
    dataSetFilePath = "./heartDisease/5_CardiacPredictionLessDimensions.xlsx"
    dataSetName = "CardiacPrediction"
elif (dataSetIndex == 6):
    dataSetFilePath = "./heartDisease/6_CardiacPredictionFewDimensions.xlsx"
    dataSetName = "CardiacPrediction"
else:
    dataSetFilePath = ""
    dataSetName = ""

if(dataSetIndex==4 or dataSetIndex==5 or dataSetIndex==6):
    #fileData = pd.read_excel(dataSetFilePath, sheet_name='CoroHeartDis')
    fileData = pd.read_excel(dataSetFilePath)
else:
    fileData = pd.read_csv(dataSetFilePath)

print("Shape of fileData: {}".format(fileData.shape))
print("Column Headings: {}".format(fileData.__dataframe__().column_names()))
print("Number of Records: {}".format(fileData.__dataframe__().num_rows()))


missingValues = fileData.isnull().any().sum()
print(f"\nNumber of Missing Values: {missingValues}")

num_rows_before = fileData.shape[0] 
fileData.drop_duplicates(inplace=True) 
num_rows_after = fileData.shape[0] 
num_duplicates_removed = num_rows_before - num_rows_after
print(f"Number of duplicate records removed: {num_duplicates_removed}")
fileData = fileData.dropna()
print("Shape of fileData: {}".format(fileData.shape))   
fileData = fileData.fillna(0) 

print("Shape of fileData End: {}".format(fileData.shape))



finalResultTable = [ ['Index', 'Method', 'Accuracy %','Recall %','Precision %','F1 Score','AUC'], ]  

X = fileData.drop(fileData.__dataframe__().column_names()[-1], axis=1)  # Features
Y = fileData[fileData.__dataframe__().column_names()[-1]]  # Labels

columns = fileData.__dataframe__().column_names() 
totalRecords = (fileData.__dataframe__().num_rows())

print("\n")
print("columns of x:: {} \n\n and features of X: {}".format(len(X.columns), X.columns))

dataSetResultDirectory = "./"
dataSetResultDirectory += ("DatasetResults_PyTorch_" + dataSetName)
dataSetResultDirectory += "/"
if not os.path.isdir(dataSetResultDirectory):
    os.makedirs(dataSetResultDirectory)

dataSetName += " {}".format(fileData.shape)



In [ ]:
print("Shape of fileData: {} , target Len:{}".format(fileData.shape, len(Y)))
print("X: {} , Y:{}".format(X.shape, Y.shape))
#print("\n\nX: head:: \n{}".format(X.head()))
#print("\n\nY: head::\n {}".format(Y.head()))

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, stratify=Y, random_state=42)
 

print("Target Column Name:: {} \n".format(fileData.__dataframe__().column_names()[-1]))


rus = RandomUnderSampler(random_state=42)
smote = SMOTE(random_state=42)

'''
# Undersample the majority class
X_train, y_train = rus.fit_resample(X_train, y_train)
X_test, y_test = rus.fit_resample(X_test, y_test)
'''

# Oversample the minority class using SMOTE
X_train, y_train = smote.fit_resample(X_train, y_train)
#X_test, y_test = rus.fit_resample(X_test, y_test)
X_test, y_test = smote.fit_resample(X_test, y_test) 


scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
#Y_train_scaled = scaler.transform(y_train)
#Y_test_scaled = scaler.transform(y_test)

 


print("\n X Train: Shape:: {}".format(X_train.shape))
print(" X Test: Shape:: {}".format(X_test.shape))  
 
 
positiveClass =  0
negativeClass = 0
for i in y_train:
    if(i == 0):
        negativeClass += 1
    if(i == 1):
        positiveClass += 1
print("Train DataSet Positive Class Records:: {}".format(positiveClass)) 
print("Train DataSet Negative Class Records:: {}".format(negativeClass)) 
print("Train DataSet Total Records:: {}".format(positiveClass + negativeClass)) 

print("\n\n") 


positiveClass =  0
negativeClass = 0
for i in y_test:
    if(i == 0):
        negativeClass += 1
    if(i == 1):
        positiveClass += 1
print("Test DataSet Positive Class Records:: {}".format(positiveClass)) 
print("Test DataSet Negative Class Records:: {}".format(negativeClass)) 
print("Test DataSet Total Records:: {}".format(positiveClass + negativeClass)) 



In [ ]:
def makeCorrelationPic(correlationMatrix, numberOfTopFeatures, targetColumnName):     
    correlation_values = correlationMatrix.abs()
    sorted_correlation = correlation_values.unstack().sort_values(ascending=False)
    sorted_correlation = sorted_correlation[sorted_correlation != 1.0]

    num_features = numberOfTopFeatures  # Number of top features to display
    top_features = sorted_correlation.head(num_features)
    print("Top", num_features, "features based on correlation:")
    print(top_features)
 
    top_features = correlationMatrix.abs().nlargest(numberOfTopFeatures, targetColumnName)[targetColumnName].index
    top_correlation_matrix = correlationMatrix.loc[top_features, top_features]

    mplot.figure(figsize=(10, 8))
    sns.heatmap(top_correlation_matrix, annot=True, cmap='coolwarm')
    # Set the title of the plot
    mplot.title('Correlation Heatmap ({})'.format(dataSetName))
    
    picturePath = "Correlation_Matrix_DateSetName_{}.png".format(dataSetName)
    mplot.savefig(picturePath,  dpi=300, bbox_inches='tight')
    mplot.show()
    mplot.close()

def makeConfusionMatrixPic(method, dataSet, classifierObj , X_test, y_test, predicted_Y):
    display = ConfusionMatrixDisplay.from_estimator(classifierObj, X_test, y_test, display_labels=['Healthy', "Heart Disease"], cmap=mplot.cm.Blues) #, normalize="true"
    display.ax_.set_title("Confusion Matrix ({} Model)".format(method))
    display.ax_.set_xlabel('\nPredicted Values')
    display.ax_.set_ylabel('Actual Values ')


    accuracyString ="Accuracy {}: {:.2f}".format(method, accuracy_score(y_test, predicted_Y)*100.0 ) 
    recallString =  'Recall {}: {:.2f}'.format(method, recall_score(y_test, predicted_Y) * 100.0)
    precisionString = 'Precision {}: {:.2f}'.format(method, precision_score(y_test, predicted_Y) * 100.0) 
    dataSetString = "Dataset: {}".format(dataSet)

    
    if(classifierObj.n_features_in_ > 10):
        featureListString = 'Total Features: {}'.format(classifierObj.n_features_in_) 
    else:
        featureListString = 'Features: {}'.format(classifierObj.feature_names_in_) 
    
    display.figure_.text(0.010, -0.05,  accuracyString, horizontalalignment='left', wrap=False )  
    display.figure_.text(0.010, -0.09,  recallString, horizontalalignment='left', wrap=False )      
    display.figure_.text(0.010, -0.13,  precisionString, horizontalalignment='left', wrap=False ) 
    display.figure_.text(0.010, -0.17,  dataSetString, horizontalalignment='left', wrap=False ) 
    display.figure_.text(0.010, -0.28,  featureListString, horizontalalignment='left', wrap=False ) 
 
    picturePath = "{}Confusion_Matrix_{}_{}.png".format(dataSetResultDirectory, method, dataSetName)
    mplot.savefig(picturePath,  dpi=300, bbox_inches='tight')
    #print("{} Confusion Matrix saved:: path: {}".format(method, picturePath))
    #os.startfile(picturePath)
    mplot.show()
    mplot.close()


In [ ]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)
#print(X_train_tensor)
#print(y_train_tensor) 
# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]
    
train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
test_dataset = CustomDataset(X_test_tensor, y_test_tensor)
print(train_dataset)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
print(train_loader)


In [ ]:
# Define the Attention model
class AttentionModel(nn.Module): 
    def __init__(self, input_size, hidden_size):
        super(AttentionModel, self).__init__()
        self.W_query = nn.Linear(input_size, hidden_size)
        self.W_key = nn.Linear(input_size, hidden_size)
        self.V = nn.Linear(hidden_size, 1)
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        query = self.W_query(x)
        keys = self.W_key(x)
        scores = self.V(torch.tanh(query + keys)).squeeze(2)
        attention_weights = torch.softmax(scores, dim=2)
        attended = torch.sum(x * attention_weights.unsqueeze(2), dim=2)
        # Fully connected layers
        x = self.fc1(attended)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x




# Instantiate the model
input_size = X_train.shape[1]
hidden_size = 64
model = AttentionModel(input_size, hidden_size)
# Loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        #optimizer.zero_grad()
        #print(inputs)
        #print(labels)
        
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(2))
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

# Evaluate on the test set
model.eval()
with torch.no_grad():
    predictions = []
    true_labels = []
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predictions.extend(outputs.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Convert to binary predictions
binary_predictions = [1 if pred > 0.5 else 0 for pred in predictions]

# Evaluate performance
accuracy = accuracy_score(true_labels, binary_predictions)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd

# Attention mechanism
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.attn = nn.Linear(hidden_size, 1)
    def forward(self, encoder_outputs, decoder_hidden):
        # Calculate attention weights
        scores = self.attn(encoder_outputs)
        scores = scores.view(-1, scores.size(1))
        attn_weights = F.softmax(scores, dim=1)
        # Context vector
        context_vector = torch.sum(attn_weights * encoder_outputs, dim=1)
        return context_vector, attn_weights

# LSTM with Attention
class LSTMAttention(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_size, output_dim, num_layers=1, dropout=0.5):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, dropout=dropout)
        self.attention = Attention(hidden_size)
        self.decoder = nn.Linear(hidden_size, output_dim)
    def forward(self, inputs, encoder_outputs):
        # Embed input sequence
        embedded = self.embedding(inputs)
        outputs, (hidden, _) = self.lstm(embedded)
        context_vector, attn_weights = self.attention(encoder_outputs, hidden)
        outputs = outputs + context_vector
        combined_output = torch.cat((outputs, context_vector), dim=1)
        output = self.decoder(combined_output)
        return output, attn_weights

# Custom Dataset Class
class HeartDataset(Dataset):
    def __init__(self, data_path):
        data = pd.read_excel(data_path)
        self.features = torch.tensor(data.drop('CoronaryHeartDisease', axis=1).values)
        self.target = torch.tensor(data['CoronaryHeartDisease'].values).float()
    def __len__(self):
        return len(self.features)
    def __getitem__(self, idx):
        return self.features[idx], self.target[idx]
# Train and evaluate
def train(model, train_loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for i, (features, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output, _ = model(features)
        loss = criterion(output, target)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    return running_loss / len(train_loader)
def evaluate(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for features, target in test_loader:
            output, _
 
 
# Load data

#dataSetFilePath =  CoronaryHeartDisease
#dataSetName = "CardiacPrediction"



In [ ]:

data_path = "./heartDisease/6_CardiacPredictionFewDimensions.xlsx"
dataset = HeartDataset(data_path) 
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
print(train_dataset)
# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
print(train_loader)


# Hyperparameters
input_dim = 38 # len(38)
embedding_dim = 64
hidden_size = 128
output_dim = 1
num_layers = 2
learning_rate = 0.01
epochs = 10
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Model and optimizer
model = LSTMAttention(input_dim, embedding_dim, hidden_size, output_dim, num_layers).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)





In [ ]:

# Train model
for epoch in range(epochs):
    running_loss = 0.0
    for i, (features, target) in enumerate(train_loader):
        features, target = features.to(device), target.to(device)
        optimizer.zero_grad()
        output, attn_weights = model(features)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 100 == 0:
            print(f"Epoch {epoch+1}/{epochs} | Step {i+1}/{len(train_loader)} | Loss: {running_loss/100:.4f}")
            running_loss = 0.0

# Evaluate model
model.eval() 

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':16:8'
 
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_size, 1)

    def forward(self, encoder_outputs, decoder_hidden):
        scores = self.attn(encoder_outputs).view(-1, encoder_outputs.size(1))
        attn_weights = F.softmax(scores, dim=1)
        context_vector = torch.sum(attn_weights * encoder_outputs, dim=1)
        return context_vector, attn_weights

class LSTMAttention(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_size, output_dim, num_layers=1, dropout=0.5):
        super(LSTMAttention, self).__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, dropout=dropout)
        self.attention = Attention(hidden_size)
        self.decoder = nn.Linear(hidden_size, output_dim)
    '''
    def forward(self, inputs, encoder_outputs):
        embedded = self.embedding(inputs)
        outputs, (hidden, _) = self.lstm(embedded)
        context_vector, attn_weights = self.attention(encoder_outputs, hidden)
        outputs = outputs + context_vector
        combined_output = torch.cat((outputs, context_vector), dim=1)
        output = self.decoder(combined_output)
        return output, attn_weights
    
    '''
    def forward(self, inputs):
        embedded = self.embedding(inputs.long())
        outputs, (hidden, _) = self.lstm(embedded)
        context_vector, attn_weights = self.attention(outputs, hidden)
        outputs = outputs + context_vector
        combined_output = torch.cat((outputs, context_vector), dim=1)
        output = self.decoder(combined_output)
        return output, attn_weights


class HeartDataset(Dataset):
    def __init__(self, data_path):
        data = pd.read_excel(data_path)
        self.features = torch.tensor(data.drop('CoronaryHeartDisease', axis=1).values)
        self.target = torch.tensor(data['CoronaryHeartDisease'].values).float()
        print("dataset features length: {}".format(len(self.features)))
        print("dataset Target length: {}".format(len(self.target)))

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.target[idx]

def train(model, train_loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for i, (features, target) in enumerate(train_loader):
        #features, target = features.to(device), target.to(device)
        optimizer.zero_grad()
        output, _ = model(features)
        loss = criterion(output, target)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    return running_loss / len(train_loader)

def evaluate(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for features, target in test_loader:
            #features, target = features.to(device), target.to(device)
            output, _ = model(features)
            loss = criterion(output, target)
            running_loss += loss.item()
    return running_loss / len(test_loader)

# Load data
data_path = "./heartDisease/6_CardiacPredictionFewDimensions.xlsx"
dataset = HeartDataset(data_path) 
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Hyperparameters
input_dim = dataset.__len__
print("input Dim is: {}".format(input_dim))
embedding_dim = 64
hidden_size = 128
output_dim = 1
num_layers = 2
learning_rate = 0.01
epochs = 10

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

# Model and optimizer
model = LSTMAttention(input_dim, embedding_dim, hidden_size, output_dim, num_layers) #.to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train model
for epoch in range(epochs):
    train_loss = train(model, train_loader, optimizer, criterion)
    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f}")

# Evaluate model
test_loss = evaluate(model, test_loader, criterion)
print(f"Test Loss: {test_loss:.4f}")


dataset features length: 37079
dataset Target length: 37079
input Dim is: <bound method HeartDataset.__len__ of <__main__.HeartDataset object at 0x00000226AC5161A0>>


TypeError: empty() received an invalid combination of arguments - got (tuple, dtype=NoneType, device=NoneType), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, torch.memory_format memory_format, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [ ]:

# Define a simple neural network
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(10, 5)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(5, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# Instantiate the model
model = SimpleModel()
# Define a loss function (e.g., binary cross-entropy for binary classification)
criterion = nn.BCELoss()
# Define an optimizer (e.g., Adam optimizer)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import torch
# Example training loop
for epoch in range(10):
    for inputs, labels in train_loader:
        # Zero the gradients
        optimizer.zero_grad()
        
        print(inputs)
        # Forward pass
        outputs = model(inputs)
        # Calculate the loss
        loss = criterion(outputs, labels)
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
    # Print or log the loss after each epoch
    print(f'Epoch {epoch + 1}/{10}, Loss: {loss.item()}')


In [ ]:
# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Define the Attention Mechanism Sequential Model
model = nn.Sequential(
    nn.Embedding(num_embeddings=100, embedding_dim=64),
    nn.LSTM(input_size=64, hidden_size=64, batch_first=True),
    # Attention mechanism components
    nn.Linear(64, 64),
    nn.Linear(64, 64),
    nn.Linear(64, 1),
    nn.Tanh(),
    # Final layer
    nn.Linear(64, 1),
    nn.Sigmoid()
) 

# Initialize the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

'''
model1 = nn.Sequential() 
model1.add(nn.Embedding(num_embeddings=100, embedding_dim=64))
model1.add(nn.LSTM(input_size=64, hidden_size=64, batch_first=True))
model1.add(nn.Linear(64, 64))
model1.add(nn.Linear(64, 64))
model1.add(nn.Linear(64, 1))
model1.add(nn.Tanh())
model1.add(nn.Linear(64, 32))
model1.add(nn.Linear(32, 1))
model1.add(nn.Sigmoid()) 
'''
model1 = nn.Sequential( nn.Embedding(num_embeddings=100, embedding_dim=64) ,
                        nn.LSTM(input_size=64, hidden_size=64, batch_first=True),
                        nn.Linear(64, 64),
                        nn.Linear(64, 64),
                        nn.Linear(64, 1),
                        nn.Tanh(),
                        nn.Linear(64, 32),
                        nn.ReLU(),
                        nn.Linear(32, 1),
                        nn.Sigmoid()
                        ) 



model1.compile(optimizer= nn.optimizers.Adam(model1.parameters(), lr=0.001), loss=nn.BCELoss, metrics=['accuracy' ])


In [ ]:

# Evaluation
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        predictions.extend(outputs.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Convert to binary predictions (assuming binary classification)
binary_predictions = [1 if pred > 0.5 else 0 for pred in predictions]

# Draw confusion matrix
cm = confusion_matrix(true_labels, binary_predictions)
mplot.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
mplot.xlabel("Predicted")
mplot.ylabel("True")
mplot.show()

# Print classification report
print(classification_report(true_labels, binary_predictions))